In [2]:
# load src/
import sys
sys.path.append('../')
import src.dataset

In [3]:
X_train, X_valid, X_test = src.dataset.load_datasets()

In [4]:
import numpy as np
a = np.load("../data/data_train.csv.0" + ".npy", allow_pickle=True).tolist()
b = np.load("../data/data_train.csv.1" + ".npy", allow_pickle=True).tolist()

In [5]:
a, b

(<10000x1000 sparse matrix of type '<class 'numpy.int64'>'
 	with 231394 stored elements in Compressed Sparse Row format>,
 <10000x1000 sparse matrix of type '<class 'numpy.int64'>'
 	with 231391 stored elements in Compressed Sparse Row format>)

In [6]:
c = a + b

In [7]:
c

<10000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 462785 stored elements in Compressed Sparse Row format>

In [8]:
m, n = 10000, 1000
U_0 = np.random.normal(size=(m, 10))
V_0 = np.random.normal(size=(10, n))

In [9]:
def compute_loss(a, u, v):
    losses = np.square(a - np.matmul(u, v))
    mean_loss = np.mean(losses[a.nonzero()])
    return mean_loss

In [10]:
mean_loss = compute_loss(X_train, U_0, V_0)
print("Initial loss:", mean_loss)

Initial loss: 26.25597369493196


In [11]:
valid_indices = set(zip(X_train.nonzero()[0], X_train.nonzero()[1]))
print(list(valid_indices)[0])

(1151, 604)


In [12]:
(0, 0) in valid_indices, (0, 9) in valid_indices

(False, True)

In [31]:
from tqdm.auto import tqdm

U, V = U_0.copy(), V_0.copy()
A = X_train.todense()
valid_indices = set(zip(X_train.nonzero()[0], X_train.nonzero()[1]))

alpha = 0.
initial_lr = 0.001 # 1e-4
gamma = initial_lr

U_new, V_new = U.copy(), V.copy()  # TODO: or like this?


print("Computing valid mask")
valid_mask = np.zeros_like(A)
for i in range(m):
    for j in range(n):
        if (i,j) in valid_indices:
            valid_mask[i, j] = 1.

print("Training")
for it in range(1000):
    
    mean_loss = compute_loss(X_train, U, V)
    valid_loss = compute_loss(X_valid, U, V)
    print("It %d\tLoss%f\tValid%f" % (it, mean_loss, valid_loss))
    
    for i in tqdm(range(m)):
        u_i = U[i, :]
        gradient = np.zeros_like(u_i)

        #for j in range(n):  # 17-18s
        #    if (i, j) in valid_indices:
        #        v_j = V[:, j]
        #        gradient += (alpha * u_i) + np.dot(u_i, v_j) * v_j - A[i, j] * v_j

        dots = np.matmul([u_i], V)
        assert dots.shape[0] == 1 and dots.shape[1] == n, (dots.shape, n)
        dots *= valid_mask[i, :]
        # print(dots.shape, V.shape)
        dots = np.multiply(dots, V)  # TODO: is this right? 10x1000 should be right
        # print("Dots:", dots.shape)
        dots_sum = np.sum(dots,axis=-1)
        assert dots_sum.shape == (10,), dots_sum.shape
        gradient += (alpha * u_i)
        gradient += dots_sum

        A_minus = np.matmul(A[i, :], V.T)  # TODO: depends on fact that A[i,j] == 0
        # A_minus = np.ravel(A_minus)
        gradient -= np.ravel(A_minus)
        # gradient = (alpha * u_i) + np.sum(dots, axis=-1)
#         u_i -= gamma * gradient
        U_new[i, :] = u_i - gamma * gradient
    
    for j in tqdm(range(n)):
        v_j = V[:, j]
        gradient = np.zeros_like(v_j)
        # for i in range(m):
        #     if valid_mask[i, j] != 0.:
        #         u_i = U[i, :]
        #         gradient += (alpha * v_j) + np.dot(u_i, v_j) * u_i - A[i, j] * u_i

        dots = np.matmul(U, v_j.reshape(-1,1))
        assert dots.shape[0] == m and dots.shape[1] == 1, (dots.shape, m)
        dots *= valid_mask[:, j]
        # print(dots.shape, V.shape)
        dots = np.multiply(dots, U)  # TODO: is this right? 10x1000 should be right
        # print("Dots:", dots.shape)
        dots_sum = np.sum(dots, axis=0)
        assert dots_sum.shape == (10,), dots_sum.shape
        gradient += (alpha * v_j)
        gradient += dots_sum

        A_minus = np.matmul(U.T, A[:, j])  # TODO: depends on fact that A[i,j] == 0
        assert A_minus.shape == (10,1), A_minus.shape
        # A_minus = np.ravel(A_minus)
        gradient -= np.ravel(A_minus)

        V_new[:, j] = v_j - gamma * gradient
    
    if gamma > 5e-4:
        gamma *= 0.1

    U, V = U_new, V_new
    U_new, V_new = U.copy(), V.copy()

168
 80%|████████  | 8009/10000 [00:03<00:01, 1742.50it/s]


KeyboardInterrupt: 

In [88]:
alpha = 2e-4

for it in range(90, 100):
    
    mean_loss = compute_loss(X_train, U, V)
    print("It %d\tLoss%f" % (it, mean_loss))
    
    for i in tqdm(range(m)):
        u_i = U[i, :]
        gradient = np.zeros_like(u_i)
        for j in range(n):
            if (i, j) in valid_indices:
                v_j = V[:, j]
                gradient += (alpha * u_i) + np.dot(u_i, v_j) * v_j - A[i, j] * v_j
#         u_i -= gamma * gradient
        U_new[i, :] = u_i - gamma * gradient
    
    for j in tqdm(range(n)):
        v_j = V[:, j]
        gradient = np.zeros_like(v_j)
        for i in range(m):
            if (i, j) in valid_indices:
                u_i = U[i, :]
                gradient += (alpha * v_j) + np.dot(u_i, v_j) * u_i - A[i, j] * u_i
        V_new[:, j] = v_j - gamma * gradient

    U, V = U_new, V_new
    U_new, V_new = U.copy(), V.copy()

It 90	Loss1.075602


It 91	Loss1.071095


It 92	Loss1.066846


It 93	Loss1.062837


It 94	Loss1.059052


It 95	Loss1.055475


It 96	Loss1.052092


It 97	Loss1.048890


It 98	Loss1.045856


It 99	Loss1.042981


In [32]:
mean_loss = compute_loss(X_train, U, V)
print("Training loss:", mean_loss)
mean_loss = compute_loss(X_valid, U, V)
print("Validation loss:", mean_loss)
mean_loss = compute_loss(X_train + X_valid, U, V)
print("All loss:", mean_loss)

Training loss: 0.9996696194161337
Validation loss: 1.057115377928021
All loss: 1.0119394503043178


In [34]:
# X_submit = np.multiply(np.matmul(U, V), X_test)
U_bak, V_bak = U.copy(), V.copy()

In [1]:
import pandas as pd
def export_and_save(target, preds):
    target_rows, target_cols = target.nonzero()
    ids = [f"r{row+1}_c{col+1}" for row, col in zip(target_rows, target_cols)]
    scores = [preds[row, col] for row, col in zip(target_rows, target_cols)]
    df = pd.DataFrame({"Id": ids, "Prediction": scores})
    df.to_csv("preds.csv", index=False)

In [37]:
export_and_save(X_test, np.matmul(U, V))

In [30]:
import pickle
with open("results.pkl", "wb") as fp:
    pickle.dump([U_bak, V_bak,    100], fp)
